In [1]:
import yfinance as yf

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

data = pdr.get_data_yahoo("AAPL", start="2013-01-01", end="2020-01-01")


[*********************100%***********************]  1 of 1 completed


In [4]:
data=data.drop(['Adj Close','Volume'], axis=1)


In [5]:
def momentum(df):
    n = len(df)
    arr = []
    for i in range(0,5):
        arr.append('N')
    for j in range(5,n):
        momentum = df.Close[j] - df.Close[j-5] #Equation for momentum
        arr.append(momentum)
    return arr

momentum = momentum(data)

data['Momentum'] = momentum

In [6]:
data_pctchange=data.Close.pct_change()
data['Return'] = data_pctchange

In [7]:
def ROI(df,n):
    m = len(df)
    arr = []
    for i in range(0,n):
        arr.append('N')
    for j in range(n,m):
        roi= (df.Close[j] - df.Close[j-n])/df.Close[j-n] 
        arr.append(roi)
    return arr


ROI10=ROI(data,10)
ROI20=ROI(data,20)
ROI30=ROI(data,30)

data['10 Day ROI']=ROI10
data['20 Day ROI']=ROI20
data['30 Day ROI']=ROI30

In [8]:

def RSI(df,period):
    n = len(df)
    arr = []
    for i in range(0,period):
        arr.append('N')
    for j in range(period,n):
        total_upwards = 0
        total_downwards = 0
       
        for k in range(j,j-period,-1):
            if(df.Close[k-1] > df.Close[k]):
                total_downwards = total_downwards + (df.Close[k-1] - df.Close[k])    
        avg_down = total_downwards / period
        for l in range(j,j-period,-1):
            if(df.Close[l] > df.Close[l-1]):
                total_upwards = total_upwards + (df.Close[l] - df.Close[l-1])
        avg_up = total_upwards / period
        RS = avg_up / avg_down
        RSI  = 100 - (100/(1+RS))
        arr.append(RSI)
    return arr

RSI_14 = RSI(data,14)
RSI_10 = RSI(data,10)
RSI_30 = RSI(data,30)

data['10_day_RSI'] = RSI_10
data['14_day_RSI'] = RSI_14
data['30_day_RSI'] = RSI_30

In [9]:
def EMA(df, n):
    m = len(df)
    arr = []
    arr.append('N')
    prevEMA = df.Close[0]
    for i in range(1,m):
        close = df.Close[i]
        EMA = ((2/(n+1))*close) + ((1-(2/(n+1)))*prevEMA)
        arr.append(EMA)
        prevEMA = EMA
    return arr

EMA_12 = EMA(data, 12)
EMA_26 = EMA(data, 26)

data['EMA_12'] = EMA_12
data['EMA_26'] = EMA_26

In [10]:

def clas(df):
    n = len(df)
    arr = []
    for i in range(0,len(df)-1):
        if (100*((df.Close[i+1]-df.Open[i+1])/df.Open[i+1]))>=.3:
            arr.append(1)
        else:
            arr.append(0)
    arr.append('N')
    return arr

clas=clas(data)

data['Class'] = clas

In [11]:
def MACD(df):
    n = 12
    m2 = 26
    arr = []
    arr.append('N')
    ema_12 = EMA(df,n)
    ema_26 = EMA(df,m2)
    m = len(df)
    for i in range(1,m):
        arr.append(ema_12[i] - ema_26[i])
    return arr

MACD = MACD(data)

data['MACD_12_26'] = MACD

In [12]:

def SRSI(df,n):
    m = len(df)
    arr = []
    list_RSI = RSI(df,n)
    for i in range(0,n):
        arr.append('N')
    for j in range(n,n+n):
        last_n = list_RSI[n:j]
        if(not(last_n == []) and not(max(last_n) == min(last_n))):
            SRSI = (list_RSI[j] - min(last_n)) / (max(last_n)- min(last_n))
            if SRSI > 1:
                arr.append(1)
            else:
                arr.append(SRSI)
        else:
            arr.append(0)
    for j in range(n+n,m):
        last_n = list_RSI[2*n:j]
        if(not(last_n == []) and not(max(last_n) == min(last_n))):
            SRSI = (list_RSI[j] - min(last_n)) / (max(last_n)- min(last_n))
            if SRSI > 1:
                arr.append(1)
            else:
                arr.append(SRSI)
        else:
            arr.append(0)
    return arr

SRSI_10 = SRSI(data,10)
SRSI_14 = SRSI(data,14)
SRSI_30 = SRSI(data,30)

data['SRSI_10'] = SRSI_10
data['SRSI_14'] = SRSI_14
data['SRSI_30'] = SRSI_30

In [13]:

def Williams(df,n):
    m = len(df)
    arr = []
    for i in range(0,n-1):
        arr.append('N')
    for j in range(n-1,m):
        maximum = max(data.High[(j-n+1):j+1])
        minimum = min(data.Low[(j-n+1):j+1])
        val = (-100)*(maximum-df.Close[j])/(maximum-minimum)
        arr.append(val)
    return arr


williams = Williams(data,14)

data['Williams'] = williams

In [14]:

def TR(df,n):
    high = df.High[n]
    low = df.Low[n]
    close = df.Close[n-1]
    l_max = list()
    l_max.append(high)
    l_max.append(close)
    l_min = list()
    l_min.append(low)
    l_min.append(close)
    return (max(l_max) - min(l_min))

def ATR(df,n):
    m = len(df)
    arr = []
    prevEMA = TR(df,n+1)
    for i in range(0,n):
        arr.append('N')
    for j in range(n,m):
        TR_ = TR(df,j)
        EMA = ((2/(n+1))*TR_) + ((1-(2/(n+1)))*prevEMA)
        arr.append(EMA)
        prevEMA = EMA
    return arr

ATR = ATR(data,14)  

data['ATR_14'] = ATR

In [15]:

import numpy as np
def CCI(df,n):
    m = len(df)
    arr = []
    tparr = []
    for i in range(0,n-1):
        arr.append('N')
        tp = (df.High[i]+df.Low[i]+df.Close[i])/3
        tparr.append(tp)
    for j in range(n-1,m):
        tp = (df.High[j]+df.Low[j]+df.Close[j])/3
        tparr.append(tp) 
        tps = np.array(tparr[(j-n+1):(j+1)])
        val = (tp-tps.mean())/(0.015*tps.std())
        arr.append(val)
    return arr

cci = CCI(data,20) 

data['CCI'] = cci

In [16]:
data.shape

(1762, 22)

In [17]:
def normalize(df):
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())

In [18]:
#Naive Bayes
def positivevalues(df):
    for column in df:
        if (df[column].min())<0:
            df[column]=(df[column]-df[column].min())

In [19]:
#Cleaning data:
newdata=data.drop(data.index[0:30])

newdata=newdata.drop(newdata.index[-1])

newdata=newdata.drop(['High','Low'], axis=1)

newdata=newdata.drop(['Class'], axis=1)

newdata.head()

,Open,Close,Momentum,Return,10 Day ROI,20 Day ROI,30 Day ROI,10_day_RSI,14_day_RSI,30_day_RSI,EMA_12,EMA_26,MACD_12_26,SRSI_10,SRSI_14,SRSI_30,Williams,ATR_14,CCI
Date,,,,,,,,,,,,,,,,,,,
2013-02-14,66.360001,66.655716,-0.232857,-0.000899,0.0243694,-0.0780493,-0.150156,58.5254,65.4805,35.248,66.8549,68.5803,-1.7254,0.649717,1,0,-37.3879,2.46554,-4.25178
2013-02-15,66.978569,65.737144,-2.11714,-0.013781,0.0144174,-0.0845866,-0.151153,54.6942,56.2417,35.3112,66.683,68.3697,-1.68676,0.572106,0.691756,0,-57.7084,2.33185,-11.5883
2013-02-19,65.871429,65.712860,-2.84857,-0.000369,0.0399485,-0.08002,-0.127154,65.0948,51.1547,37.3082,66.5337,68.1729,-1.6392,0.782802,0.522032,1,-58.1043,2.19008,-22.6204
2013-02-20,65.384285,64.121429,-2.72143,-0.024218,-0.0196357,-0.110783,-0.143253,41.699,45.2602,36.2056,66.1626,67.8728,-1.71021,0.308848,0.325366,0.46478,-84.0475,2.11121,-46.817
2013-02-21,63.714287,63.722858,-2.99286,-0.006216,-0.0246857,-0.132196,-0.150863,40,44.4938,35.5071,65.7872,67.5654,-1.77815,0.27443,0.299796,0.12576,-90.5449,1.95067,-87.7944


In [20]:
#Normalize data
normalize(newdata)

In [21]:
X=newdata
y=clas[30:-1]

In [22]:
#Split test and train 70%/30%
X_train=X.drop(X.index[1211:])
X_test=X.drop(X.index[0:1211])
y_train=y[0:1211]
y_test=y[1211:]

In [23]:
#Logistic Regression and run a fit to train the model
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
#Predict the y test 
y_pred_LR=LR.predict(X_test)

In [25]:
#accuracy score predicted y
from sklearn import metrics
print (metrics.accuracy_score(y_test, y_pred_LR)) 

0.5461538461538461


In [26]:
#Naive Bayes and run a fit to train the model
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()
GNB.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [27]:
#Predict the y test
y_pred=GNB.predict(X_test)

In [28]:
#accuracy score of predicted y
from sklearn import metrics
print (metrics.accuracy_score(y_test, y_pred)) 

0.5173076923076924


In [31]:
!pip install pyfolio

In [44]:
import pyfolio as pf
